<a href="https://colab.research.google.com/github/kghilothia06/Stock-Price-prediction/blob/main/Copy_of_Stock_Prices_Prediction_LSTM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
import plotly.express as px
from copy import copy
from scipy import stats

# EDA & Data Visualization

In [2]:
stocks_price_df = pd.read_csv('/content/drive/MyDrive/stock.csv')

In [3]:
stocks_volume_df = pd.read_csv('/content/drive/MyDrive/stock_volume.csv')

In [4]:
stocks_price_df.head()

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
0,2012-01-12,60.198570,75.510002,30.120001,12.13,175.929993,180.550003,28.250000,313.644379,1295.500000
1,2012-01-13,59.972858,74.599998,30.070000,12.35,178.419998,179.160004,22.790001,311.328064,1289.089966
2,2012-01-17,60.671429,75.239998,30.250000,12.25,181.660004,180.000000,26.600000,313.116364,1293.670044
3,2012-01-18,61.301430,75.059998,30.330000,12.73,189.440002,181.070007,26.809999,315.273285,1308.040039
4,2012-01-19,61.107143,75.559998,30.420000,12.80,194.449997,180.520004,26.760000,318.590851,1314.500000


In [5]:
stocks_volume_df

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
0,2012-01-12,53146800,3934500,26511100,17891100,5385800,6881000,729300,3764400,4019890000
1,2012-01-13,56505400,4641100,22096800,16621800,4753500,5279200,5500400,4631800,3692370000
2,2012-01-17,60724300,3700100,23500200,15480800,5644500,6003400,4651600,3832800,4010490000
3,2012-01-18,69197800,4189500,22015000,18387600,7473500,4600600,1260200,5544000,4096160000
4,2012-01-19,65434600,5397300,25524000,14022900,7096000,8567200,1246300,12657800,4465890000
...,...,...,...,...,...,...,...,...,...,...
2154,2020-08-05,30498000,46551000,22991700,18914200,3930000,3675400,4978000,1979500,4732220000
2155,2020-08-06,50607200,32921600,21908700,35867700,3940600,3417100,5992300,1995400,4267490000
2156,2020-08-07,49453300,19301600,30398500,34530300,3929600,3651000,8883500,1576600,4104860000
2157,2020-08-10,53100900,35857700,35514400,71219700,3167300,3968300,7522300,1289300,4318570000


Sort both datasets by date

In [6]:
stocks_price_df.sort_values(by=['Date'])

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
0,2012-01-12,60.198570,75.510002,30.120001,12.130000,175.929993,180.550003,28.250000,313.644379,1295.500000
1,2012-01-13,59.972858,74.599998,30.070000,12.350000,178.419998,179.160004,22.790001,311.328064,1289.089966
2,2012-01-17,60.671429,75.239998,30.250000,12.250000,181.660004,180.000000,26.600000,313.116364,1293.670044
3,2012-01-18,61.301430,75.059998,30.330000,12.730000,189.440002,181.070007,26.809999,315.273285,1308.040039
4,2012-01-19,61.107143,75.559998,30.420000,12.800000,194.449997,180.520004,26.760000,318.590851,1314.500000
...,...,...,...,...,...,...,...,...,...,...
2154,2020-08-05,440.250000,174.279999,29.850000,16.719999,3205.030029,125.449997,1485.020020,1473.609985,3327.770020
2155,2020-08-06,455.609985,172.199997,29.840000,18.459999,3225.000000,126.120003,1489.579956,1500.099976,3349.159912
2156,2020-08-07,444.450012,170.020004,30.020000,19.030001,3167.459961,124.959999,1452.709961,1494.489990,3351.280029
2157,2020-08-10,450.910004,179.410004,30.200001,21.650000,3148.159912,127.110001,1418.569946,1496.099976,3360.469971


In [7]:
stocks_volume_df.sort_values(by=['Date'])

,Date,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
0,2012-01-12,53146800,3934500,26511100,17891100,5385800,6881000,729300,3764400,4019890000
1,2012-01-13,56505400,4641100,22096800,16621800,4753500,5279200,5500400,4631800,3692370000
2,2012-01-17,60724300,3700100,23500200,15480800,5644500,6003400,4651600,3832800,4010490000
3,2012-01-18,69197800,4189500,22015000,18387600,7473500,4600600,1260200,5544000,4096160000
4,2012-01-19,65434600,5397300,25524000,14022900,7096000,8567200,1246300,12657800,4465890000
...,...,...,...,...,...,...,...,...,...,...
2154,2020-08-05,30498000,46551000,22991700,18914200,3930000,3675400,4978000,1979500,4732220000
2155,2020-08-06,50607200,32921600,21908700,35867700,3940600,3417100,5992300,1995400,4267490000
2156,2020-08-07,49453300,19301600,30398500,34530300,3929600,3651000,8883500,1576600,4104860000
2157,2020-08-10,53100900,35857700,35514400,71219700,3167300,3968300,7522300,1289300,4318570000


In [8]:
stocks_price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2159 entries, 0 to 2158
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2159 non-null   object 
 1   AAPL    2159 non-null   float64
 2   BA      2159 non-null   float64
 3   T       2159 non-null   float64
 4   MGM     2159 non-null   float64
 5   AMZN    2159 non-null   float64
 6   IBM     2159 non-null   float64
 7   TSLA    2159 non-null   float64
 8   GOOG    2159 non-null   float64
 9   sp500   2159 non-null   float64
dtypes: float64(9), object(1)
memory usage: 168.8+ KB


In [9]:
stocks_volume_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2159 entries, 0 to 2158
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    2159 non-null   object
 1   AAPL    2159 non-null   int64 
 2   BA      2159 non-null   int64 
 3   T       2159 non-null   int64 
 4   MGM     2159 non-null   int64 
 5   AMZN    2159 non-null   int64 
 6   IBM     2159 non-null   int64 
 7   TSLA    2159 non-null   int64 
 8   GOOG    2159 non-null   int64 
 9   sp500   2159 non-null   int64 
dtypes: int64(9), object(1)
memory usage: 168.8+ KB


In [10]:
stocks_price_df.describe()

,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
count,2159.000000,2159.000000,2159.000000,2159.000000,2159.000000,2159.000000,2159.000000,2159.000000,2159.000000
mean,140.819823,189.942700,35.162899,23.105743,915.665665,161.853001,259.600815,783.712512,2218.749554
std,70.827601,103.678586,3.207490,6.963847,697.838905,25.561938,210.988003,334.448057,537.321727
min,55.790001,67.239998,26.770000,7.140000,175.929993,94.769997,22.790001,278.481171,1278.040039
25%,89.165714,124.015000,33.040001,18.545000,316.490005,142.769997,184.595001,527.214416,1847.984985
50%,116.599998,142.419998,34.930000,23.780001,676.010010,156.949997,231.960007,737.599976,2106.629883
75%,175.019997,297.044998,37.419998,28.430000,1593.645019,185.974998,307.350006,1079.744995,2705.810059
max,455.609985,440.619995,43.470001,38.029999,3225.000000,215.800003,1643.000000,1568.489990,3386.149902


In [11]:
stocks_volume_df.describe()

,AAPL,BA,T,MGM,AMZN,IBM,TSLA,GOOG,sp500
count,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03,2.159000e+03
mean,5.820332e+07,6.419916e+06,2.832131e+07,9.845582e+06,4.102673e+06,4.453090e+06,7.001302e+06,2.498238e+06,3.680732e+09
std,4.568141e+07,9.711873e+06,1.428911e+07,7.295753e+06,2.290722e+06,2.462811e+06,5.781208e+06,1.928407e+06,8.622717e+08
min,1.136200e+07,7.889000e+05,6.862400e+06,9.507000e+05,8.813000e+05,1.193000e+06,3.649000e+05,7.900000e+03,1.248960e+09
25%,2.769930e+07,3.031850e+06,2.002150e+07,5.796450e+06,2.675700e+06,3.111250e+06,3.433450e+06,1.325400e+06,3.211890e+09
50%,4.209420e+07,3.991000e+06,2.485930e+07,7.899800e+06,3.494800e+06,3.825000e+06,5.581100e+06,1.813900e+06,3.526890e+09
75%,7.182480e+07,5.325900e+06,3.210565e+07,1.104055e+07,4.768150e+06,4.937300e+06,8.619550e+06,3.245350e+06,3.933290e+09
max,3.765300e+08,1.032128e+08,1.950827e+08,9.009820e+07,2.385610e+07,3.049020e+07,6.093880e+07,2.497790e+07,9.044690e+09


Function to normalize stock prices based on their initial price

In [12]:
def normalize(df):
  df1 = df.copy()

  for col in df.columns:
    if col!='Date':
      df1[col][0] = 1
      for i in range(1,len(df)):
        df1[col][i] = df[col][i]/df[col][0] 
  
  return df1


Function to plot interactive plots using Plotly Express


In [13]:
def interactive_plot(df,title):
  fig = px.line(title=title)

  for i in df.columns[1:]:
    fig.add_scatter(x=df['Date'] , y=df[i] , name=i)

  fig.show()

In [14]:
interactive_plot(stocks_price_df,'individual stock price v/s date')

In [15]:
interactive_plot(stocks_volume_df, 'individual stock volumes v/s date')

In [16]:
interactive_plot(normalize(stocks_price_df),'individual stock price v/s date (normalized)')

In [17]:
interactive_plot(normalize(stocks_volume_df), 'individual stock volumes v/s date (normalized)')

# Data Preparation for Model Training

Function to concatenate the date, stock price, and volume in one dataframe

In [18]:
def individual_stock(price_df , vol_df , name):
  return pd.DataFrame({'Date' : price_df['Date'] , 'Closing_price' : price_df[name] , 'volume' : vol_df[name]})

> Function to return the input/output (target) data for AI/ML Model

> The goal is to predict the future stock price 

> Target stock price today will be tomorrow's price 

In [19]:
def trading_window(df):
  n = 1
  df['Target'] = df[['Closing_price']].shift(-n)
  return df

# Functionality to build a LSTM model

In [20]:
from tensorflow import keras

In [21]:
def build_and_train_LSTM(price_df , volume_df , stock_name):
  #filter out the individual stock
  price_volume_df = individual_stock(price_df, volume_df, stock_name)
  #include 'target' column to it
  price_volume_target_df = trading_window(price_volume_df)

  #Perform Feature Scaling
  from sklearn.preprocessing import MinMaxScaler
  sc = MinMaxScaler(feature_range=(0,1))

  price_volume_target_scaled_df = sc.fit_transform(price_volume_target_df.drop(columns=['Date']))

  #train-test split
  X=[]
  y=[]
  
  for i in range(1,len(price_volume_target_scaled_df)):
    X.append(price_volume_target_scaled_df[i-1:i,0])
    y.append(price_volume_target_scaled_df[i,0])
  
  
  # Convert the data into array format
  X = np.asarray(X) 
  y = np.asarray(y)

  #print(X.shape)

  #Split the data
  split = int(0.7 * len(X))
  X_train = X[:split]
  y_train = y[:split]
  X_test = X[split:]
  y_test = y[split:]

  # Reshape the 1D arrays to 3D arrays to feed in the model
  X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
  X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
  #print(X_train.shape, X_test.shape)
  

  #Build the model

  inputs = keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2]))
  x = keras.layers.LSTM(150, return_sequences= True)(inputs)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150, return_sequences=True)(x)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150)(x)
  outputs = keras.layers.Dense(1, activation='linear')(x)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam', loss="mse")
  model.summary()

  hist=model.fit(X_train,y_train,epochs=20,batch_size=32,validation_split=0.2)

  #Make predictions on whole dataset of 'AAPL' stock
  y_pred = model.predict(X)

  #append y_pred to a list
  y_pred_list = []
  for i in y_pred:
    y_pred_list.append(i[0])

  # Append the closing price values to the list
  close = []
  for i in price_volume_target_scaled_df:
    close.append(i[0])

  # Create a dataframe based on the dates in the individual stock data
  df_pred = price_volume_target_df[1:][['Date']]
  
  # append closing price column
  df_pred['closing_price'] = close[1:]
  
  # append predictions column
  df_pred['Predictions'] = y_pred

  # make an interactive plot of actaul v/s predicted prices
  interactive_plot(df_pred, 'Actual v/s Predictions for ' + stock_name)


In [22]:
for col in stocks_price_df.columns[1:]:
  build_and_train_LSTM(normalize(stocks_price_df) , normalize(stocks_volume_df) , col )

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 1)]            0         
_________________________________________________________________
lstm (LSTM)                  (None, 1, 150)            91200     
_________________________________________________________________
dropout (Dropout)            (None, 1, 150)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 150)            180600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 150)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense (Dense)                (None, 1)                 151   

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 1, 1)]            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 1, 150)            91200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 150)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 1, 150)            180600    
_________________________________________________________________
dropout_3 (Dropout)          (None, 1, 150)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 151 

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1, 1)]            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 1, 150)            91200     
_________________________________________________________________
dropout_4 (Dropout)          (None, 1, 150)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 1, 150)            180600    
_________________________________________________________________
dropout_5 (Dropout)          (None, 1, 150)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 150)               180600    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 151 

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1, 1)]            0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 1, 150)            91200     
_________________________________________________________________
dropout_6 (Dropout)          (None, 1, 150)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 1, 150)            180600    
_________________________________________________________________
dropout_7 (Dropout)          (None, 1, 150)            0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 150)               180600    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 151 

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 1, 1)]            0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 1, 150)            91200     
_________________________________________________________________
dropout_8 (Dropout)          (None, 1, 150)            0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 1, 150)            180600    
_________________________________________________________________
dropout_9 (Dropout)          (None, 1, 150)            0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 150)               180600    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 151 

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 1, 1)]            0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 1, 150)            91200     
_________________________________________________________________
dropout_10 (Dropout)         (None, 1, 150)            0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 1, 150)            180600    
_________________________________________________________________
dropout_11 (Dropout)         (None, 1, 150)            0         
_________________________________________________________________
lstm_17 (LSTM)               (None, 150)               180600    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 151 

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 1, 1)]            0         
_________________________________________________________________
lstm_18 (LSTM)               (None, 1, 150)            91200     
_________________________________________________________________
dropout_12 (Dropout)         (None, 1, 150)            0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 1, 150)            180600    
_________________________________________________________________
dropout_13 (Dropout)         (None, 1, 150)            0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 150)               180600    
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 151 

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 1, 1)]            0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 1, 150)            91200     
_________________________________________________________________
dropout_14 (Dropout)         (None, 1, 150)            0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 1, 150)            180600    
_________________________________________________________________
dropout_15 (Dropout)         (None, 1, 150)            0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 150)               180600    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 151 

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 1, 1)]            0         
_________________________________________________________________
lstm_24 (LSTM)               (None, 1, 150)            91200     
_________________________________________________________________
dropout_16 (Dropout)         (None, 1, 150)            0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 1, 150)            180600    
_________________________________________________________________
dropout_17 (Dropout)         (None, 1, 150)            0         
_________________________________________________________________
lstm_26 (LSTM)               (None, 150)               180600    
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 151 